## How to generate and validate (here german) freelance contract with the help of LLM and Pydantic

### Imports

In [ ]:
from openai import OpenAI
from typing import List, Optional, Literal, Dict, Any
from pydantic import BaseModel, Field, ValidationError, condecimal
from dotenv import load_dotenv
import instructor
from enum import Enum
from datetime import datetime
from loguru import logger
import sys
import datetime



In [ ]:
load_dotenv()

In [ ]:
client = OpenAI(
    base_url = "https://llm-xxxxxthis part you need to fill yourself look how it was done in LLMEDGE repository"
)


client = instructor.patch(client, mode=instructor.Mode.MD_JSON)

### Pydantic classes

In [ ]:
Vertrag_ueber_freie_mitarbeit_v0="empty"

In [ ]:
class LegalClauses(BaseModel):
    content: str= Field(
        ...,
        description="""
        Der Inhalt der Klausel/Kapitell wird nach den Regeln der Rechtssprache formuliert. Muss die folgenden Kriterien erfüllen:

 1. Spezifität:
 - eine klare Definition der Rechte, Pflichten oder Bedingungen
 - keine Unklarheiten oder Mehrdeutigkeiten in den Aufzeichnungen

 2. Formalitäten:
 - Verwendung des offiziellen Stils
 - Ausschluss von Alltagssprache und Umgangssprache

 3. Konsistenz:
 - logische Verbindung mit anderen Klauseln
 - kein Widerspruch zu anderen Bestimmungen

 4. Eindeutigkeit:
 - Verständlichkeit im rechtlichen Kontext
 - genaue Definition der Bedingungen und des Anwendungsbereichs

 5. Formatierung:
 - akzeptable Aufteilung in Sätze oder Punkte
 - Unterscheidung wichtiger Elemente (Ausnahmen, Bedingungen, Fristen)
 - Verwendung geeigneter syntaktischer Konstruktionen (z. B. „vorausgesetzt, dass“)


        """
    )
    reasoning: str = Field(..., description="Gedanken des Assistenten zu dem Satz/Absatz. Angabe rechtlicher Gründe für die Notwendigkeit der Aufnahme einer bestimmten Klausel in den Vertrag ")

class ComplianceCheck(BaseModel):
    result: str
    reasoning: str =Field(..., description="Schrittweise Erläuterung des Compliance-Prüfungsprozesses mit Einbindung von Links zum BGB oder anderen Rechtsquellen") 

class FormattedContract(BaseModel):
    document: str= Field(..., description=f"""Ein Vertrag über freie Mitarbeit mit allen erforderlichen Daten der Vertragsparteien, Klauseln und Daten der Parteien etc., erstellt auf Basis einer Vorlage:{Vertrag_ueber_freie_mitarbeit_v0}""")



In [ ]:


class Auftraggeber(BaseModel):
    name: str = Field(..., description="Name des Auftraggebers")
    adresse: str = Field(..., description="Adresse des Auftraggebers")
    vertreter: Optional[str] = Field(None, description="Vertreter des Auftraggebers (falls vorhanden)")


class Auftragnehmer(BaseModel):
    name: str = Field(..., description="Name des Auftragnehmers")
    adresse: str = Field(..., description="Adresse des Auftragnehmers")


class Taetigkeit(BaseModel):
    beschreibung: str = Field(..., description="Beschreibung der Tätigkeit")
    startdatum: str = Field(..., description="Startdatum der Tätigkeit")


class Leistungserbringung(BaseModel):
    persoenlich: bool = Field(..., description="Erbringt der Auftragnehmer die Leistung persönlich?")
    unterauftrag: bool = Field(..., description="Darf der Auftragnehmer Unteraufträge vergeben?")
    eigene_raeume: bool = Field(..., description="Arbeitet der Auftragnehmer in eigenen Räumlichkeiten?")
    betriebsmittel: bool = Field(..., description="Stellt der Auftraggeber die Betriebsmittel zur Verfügung?")


class Verguetung(BaseModel):
    stundensatz: condecimal(gt=0) = Field(..., description="Stundensatz in Euro")
    mehrwertsteuer: bool = Field(..., description="Ist die Mehrwertsteuer enthalten?")
    abrechnung_bis: int = Field(..., description="Tag des Folgemonats, bis zu dem die Abrechnung erfolgen muss")
    zahlung_faelligkeit: str = Field(..., description="Wann die Zahlung fällig ist")


class Aufwendungen(BaseModel):
    pauschal_abgedeckt: bool = Field(..., description="Sind alle Kosten durch das Honorar abgedeckt?")
    erstattungsfaehig: bool = Field(..., description="Sind Aufwendungen erstattungsfähig?")
    vorherige_zustimmung: bool = Field(..., description="Erfordert es eine Zustimmung für hohe Ausgaben?")


class Haftung(BaseModel):
    haftungsgrenze: Optional[condecimal(gt=0)] = Field(None, description="Haftungsgrenze in Euro")
    berufshaftpflicht: bool = Field(..., description="Ist eine Berufshaftpflichtversicherung erforderlich?")


class Konkurrenz(BaseModel):
    andere_auftraggeber: bool = Field(..., description="Darf der Auftragnehmer für andere Auftraggeber tätig sein?")
    wettbewerber_erlaubt: bool = Field(..., description="Darf der Auftragnehmer für Wettbewerber tätig sein?")
    vertragsstrafe: Optional[condecimal(gt=0)] = Field(None, description="Vertragsstrafe bei Verstoß in Euro")


class Verschwiegenheit(BaseModel):
    vertragsstrafe: Optional[condecimal(gt=0)] = Field(None, description="Vertragsstrafe bei Verletzung der Geheimhaltung")


class Vertragsdauer(BaseModel):
    beginn: str = Field(..., description="Beginn des Vertrags")
    kuendigungsfrist: int = Field(..., description="Kündigungsfrist in Wochen oder Monaten")
    kuendigungsdatum: Optional[str] = Field(None, description="Enddatum des Vertrags, falls befristet")


class Gerichtsstand(BaseModel):
    erfuellungsort: str = Field(..., description="Erfüllungsort")
    gerichtsstand: str = Field(..., description="Gerichtsstand")


class FreelancerVertrag(BaseModel):
    auftraggeber: Auftraggeber
    auftragnehmer: Auftragnehmer
    taetigkeit: Taetigkeit
    leistungserbringung: Leistungserbringung
    verguetung: Verguetung
    aufwendungen: Aufwendungen
    haftung: Haftung
    konkurrenz: Konkurrenz
    verschwiegenheit: Verschwiegenheit
    vertragsdauer: Vertragsdauer
    gerichtsstand: Gerichtsstand
    legal_clauses: LegalClauses
    compliance_check: ComplianceCheck
    formatted_contract: FormattedContract


### API call

In [ ]:
def call_openai(system_prompt: str, user_prompt: str, response_model):
    return client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
        response_model=response_model  # Automatic mapping
    )

### Saving  to file funcion

In [ ]:
def save_contract_to_file(contract: FreelancerVertrag):
    # Generate the contract content using the updated template with markdown formatting
    #used template from  https://www.frankfurt-main.ihk.de/recht/mustervertraege/freie-mitarbeiter-vertrag-5196842
    # it act like a dooping or buster so contract look much more like human made after initial run
    contract_text = f"""

# Vertrag über freie Mitarbeit

(Bitte überprüfen Sie, welche Vertragsbestimmungen übernommen werden wollen. Gegebenenfalls sind Anpassungen und Ergänzungen zu empfehlen.)

Zwischen  
{contract.auftraggeber.name}  
{contract.auftraggeber.adresse}  

(ggf.: vertreten durch {contract.auftraggeber.vertreter or 'Nicht angegeben'})  
- nachfolgend „Auftraggeber“ genannt -

und

Herrn/Frau {contract.auftragnehmer.name}  
wohnhaft {contract.auftragnehmer.adresse}  
- nachfolgend „Auftragnehmer“ genannt -

wird folgendes vereinbart:

---

## § 1 Tätigkeit

Der Auftragnehmer {contract.taetigkeit.beschreibung} wird ab dem {contract.taetigkeit.startdatum} für den Auftraggeber folgende Tätigkeiten als Auftragnehmer übernehmen: __________________. Ergänzend wird im Einzelfall auf die jeweiligen Auftragsschreiben verwiesen.

Der Auftragnehmer hat die Durchführung und den Ablauf seiner Leistung selbst zu organisieren. Er unterliegt keinen Weisungen des Auftraggebers und ist in der Gestaltung seiner Tätigkeit frei. Auf besondere betriebliche Belange im Zusammenhang mit seiner Tätigkeit ist jedoch Rücksicht zu nehmen.

Der Auftragnehmer ist an keinerlei Vorgaben zum Arbeitsort oder Arbeitszeit gebunden. Projektbezogene Zeitvorgaben des Auftraggebers sind ebenso einzuhalten wie fachliche Vorgaben, soweit diese zur ordnungsgemäßen Vertragsdurchführung erforderlich sind.

Der Auftragnehmer ist ferner berechtigt, Aufträge des Auftraggebers ohne Angaben von Gründen abzulehnen.

Gegenüber den Angestellten des Auftraggebers hat der Auftragnehmer keine Weisungsbefugnis.

---

## § 2 Leistungserbringung

Der Auftragnehmer erbringt die Arbeitsleistung in der Regel {'persönlich' if contract.taetigkeit else 'durch Dritte'}. Er kann sich zur Erfüllung des Auftrags auch anderer Personen bedienen. Die Hinzuziehung eigener Mitarbeiter oder die Vergabe von Unteraufträgen erfolgt in Abstimmung mit dem Auftraggeber. Für die ordnungsgemäße Erfüllung der vertraglichen Leistungen bleibt er dem Auftraggeber gegenüber verantwortlich.

Für die steuerlichen und sozialversicherungsrechtlichen Belange hat der freie Mitarbeiter selbst Sorge zu tragen, insbesondere auch für eine angemessene Versicherung für die Altersvorsorge wie auch zum Schutz gegen Krankheiten und den Pflegefall.

Der Auftragnehmer übt seine Tätigkeit in seinen eigenen Räumlichkeiten aus. Soweit in Einzelfällen eine betriebliche Anwesenheit erforderlich wird, stellt der Auftraggeber nach jeweiliger vorheriger Absprache die entsprechenden betrieblichen Einrichtungen zur Verfügung. Der Auftraggeber stellt dem Auftragnehmer alle zur Ausübung seiner Tätigkeiten erforderlichen Informationen, Hilfsmittel und Unterlagen zur Verfügung.

---

## § 3 Vergütung

Als Vergütung wird ein Stundenhonorar von {contract.verguetung.stundensatz} € zuzüglich der jeweiligen gesetzlichen Mehrwertsteuer vereinbart. Der Auftragnehmer ist verpflichtet, jeweils bis zum Zehnten des Folgemonats eine spezifizierte Abrechnung in Form einer Rechnung zu erstellen.

Das vereinbarte pauschale Honorar wird jeweils am Monatsende fällig. Die Auszahlung erfolgt unbar.

Der Auftragnehmer wird innerhalb von 14 Tagen nach Beginn der Zusammenarbeit dem Auftraggeber ein Konto benennen, auf das das Honorar angewiesen werden kann.

---

## § 4 Aufwendungsersatz und sonstige Ansprüche

Mit der Zahlung der in diesem Vertrag vereinbarten Vergütung sind alle Ansprüche des Auftragnehmers gegen den Auftraggeber aus diesem Vertrag erfüllt.

oder

Der Auftragnehmer hat Anspruch auf Ersatz der abgerechneten und nachgewiesenen Aufwendungen, die ihm im Rahmen dieser Vereinbarung in der Ausübung seiner Tätigkeit entstehen. Das Normalmaß erheblich übersteigende Ausgaben werden jedoch nur dann ersetzt, wenn der Auftragnehmer zuvor die Zustimmung des Auftraggebers eingeholt hat. Für die Versteuerung der Vergütung hat der Auftragnehmer selbst zu sorgen.

Der Auftragnehmer wird darauf hingewiesen, dass er nach § 2 Nr. 9 SGB VI rentenversicherungspflichtig sein kann, wenn er auf Dauer und im Wesentlichen nur für einen Auftraggeber tätig ist.

---

## § 5 Haftung und Gewährleistung

Sollte der Auftraggeber auf Grund von Leistungen, die vom Auftragnehmer erbracht wurden, in Haftung genommen werden, so verpflichtet sich der Auftragnehmer gegenüber dem Auftraggeber, diesen von derlei Haftung freizustellen.

Für Schäden, die durch Zeitüberschreitung des Auftragnehmers erfolgen, ist die Haftung des Auftragnehmers auf die Höhe von {contract.haftung.haftungsgrenze} € begrenzt. Im Übrigen verpflichtet sich der Auftragnehmer zur kostenlosen Nacharbeit und Beseitigung der von ihm verursachten Mängel.

---

## § 6 Fortbildungspflicht

Der Auftragnehmer ist verpflichtet, sich im Rahmen der Durchführung dieses Vertrages auf dem Gebiet seiner Tätigkeit über den aktuellen Entwicklungsstand weiterzubilden und sich über aktuelle Veränderungen auf diesem Gebiet jederzeit auf dem Laufenden zu halten.

---

## § 7 Konkurrenz

Der Auftragnehmer darf auch für andere Auftraggeber tätig sein. Will der Auftragnehmer allerdings für einen unmittelbaren Wettbewerber des Auftraggebers tätig werden, bedarf dies der vorherigen schriftlichen Zustimmung des Auftraggebers.

Der Auftragnehmer verpflichtet sich, für jeden Fall der Zuwiderhandlung eine Vertragsstrafe in Höhe von {contract.konkurrenz.vertragsstrafe} € zu zahlen.

---

## § 8 Verschwiegenheit, Aufbewahrung und Rückgabe von Unterlagen

Die Vertragsparteien verpflichten sich, alle ihnen im Rahmen des Vertrages zugänglich gemachten, sowie bei Gelegenheit der Zusammenarbeit erlangten Informationen über Angelegenheiten der anderen Partei, die als vertraulich gekennzeichnet sind; die bei einer mündlichen Übermittlung als vertraulich bezeichnet werden; oder die aus Sicht eines objektiven Beobachters als vertraulich erkennbar sind; sowie Geschäfts- und Betriebsgeheimnisse vertraulich zu behandeln. Vertrauliche Informationen dürfen ohne schriftliche Einwilligung der anderen Vertragspartei zu einem anderen als dem zur vertragsgemäßen Aufgabenerfüllung vorgesehenen Zweck nicht verwertet, Dritten zugänglich gemacht oder sonst genutzt werden.

Die Parteien tragen dafür Sorge, dass Dritte, derer sie sich als Erfüllungsgehilfen bedienen, ebenfalls die Geheimhaltungspflicht beachten.

Für jeden Fall der schuldhaften Verletzung dieser Verpflichtungen wird eine Vertragsstrafe in Höhe von {contract.verschwiegenheit.vertragsstrafe} € vereinbart.

Weitergehender Schadensersatz sowie die Geltendmachung von Unterlassungsansprüchen bleiben vorbehalten.

---

## § 9 Vertragsdauer und Kündigung

Der Auftragnehmer nimmt die Tätigkeit am {contract.vertragsdauer.beginn} auf.

Das Vertragsverhältnis kann unter Einhaltung einer Frist von {contract.vertragsdauer.kuendigungsfrist} Wochen/Monaten zum {contract.vertragsdauer.kuendigungsdatum} gekündigt werden. Das Recht zur außerordentlichen Kündigung bleibt hiervon unberührt. Jede Kündigung bedarf zu ihrer Wirksamkeit der Schriftform.

---

## § 10 Erfüllungsort und Gerichtsstand

Erfüllungsort und Gerichtsstand ist ____________.

---

## § 12 Nebenabreden und salvatorische Klausel

Nebenabreden und Änderungen des Vertrages bedürfen zu ihrer Wirksamkeit der Schriftform. Dieses Formerfordernis kann weder mündlich noch stillschweigend aufgehoben oder außer Kraft gesetzt werden.

Die teilweise oder vollständige Unwirksamkeit einzelner Bestimmungen dieses Vertrages berührt nicht die Wirksamkeit der übrigen Regelungen des Vertrages.

---

## § 13 Vertragsaushändigung

Jede der Vertragsparteien hat eine schriftliche Ausfertigung dieses Vertrages erhalten.

---

Ort, Datum: 

Unterschrift Auftraggeber: ___________________  
Unterschrift Auftragnehmer: ___________________

    """

    # Define the file path to save the contract
        # 🔹 Get the current timestamp
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # 🔹 Format filename based on Auftraggeber's name
    property_name = contract.auftraggeber.name.replace(" ", "_")
    filename = f"{property_name}_{timestamp}.md"
    # file_path = f"{contract.auftraggeber.name}_vertrag.md"

    # Save the contract content to a markdown file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(contract_text)

    print(f"Contract saved to {filename}")



### Generating contract

In [ ]:
def generate_contract(initial_contract: FreelancerVertrag, legal_prompts, compliance_prompts, formatting_prompts, iterations=3):
    try:
        # Initial contract is already a FreelancerVertrag object
        current_contract = initial_contract
        
        for i in range(iterations):
            print(f"🔄 Iteration {i+1}: Refining the contract...")

            # Generate legal clauses
            legal_clauses = call_openai(
                legal_prompts["system"],
                f"{legal_prompts['user']} Current contract version: {current_contract.formatted_contract.document}",
                LegalClauses
            )

            # Validate compliance
            compliance_check = call_openai(
                compliance_prompts["system"],
                f"{compliance_prompts['user']} Current contract version: {current_contract.formatted_contract.document}",
                ComplianceCheck
            )

            # Format the contract
            formatted_contract = call_openai(
                formatting_prompts["system"],
                f"{formatting_prompts['user']} Current contract version: {current_contract.formatted_contract.document}",
                FormattedContract
            )

            # Update the contract with new generated sections
            current_contract = FreelancerVertrag(
                auftraggeber=current_contract.auftraggeber,
                auftragnehmer=current_contract.auftragnehmer,
                taetigkeit=current_contract.taetigkeit,
                leistungserbringung=current_contract.leistungserbringung,
                verguetung=current_contract.verguetung,
                aufwendungen=current_contract.aufwendungen,
                haftung=current_contract.haftung,
                konkurrenz=current_contract.konkurrenz,
                verschwiegenheit=current_contract.verschwiegenheit,
                vertragsdauer=current_contract.vertragsdauer,
                gerichtsstand=current_contract.gerichtsstand,
                legal_clauses=legal_clauses,
                compliance_check=compliance_check,
                formatted_contract=formatted_contract
            )

            # Save contract after each iteration
            print("\nCurrent Contract\n", current_contract)
            save_contract_to_file(current_contract)
            print("\n\nLegal Clauses\n", legal_clauses)
            print("\nCompliance Check\n", compliance_check)

        return current_contract
    except ValidationError as e:
        return f"❌ Validation Error: {e}"


### Initial input data

In [ ]:
vertrag = FreelancerVertrag(
    auftraggeber=Auftraggeber(
        name="Firma XYZ GmbH",
        adresse="Musterstraße 1, 12345 Musterstadt",
        vertreter="Max Mustermann"
    ),
    auftragnehmer=Auftragnehmer(
        name="John Doe",
        adresse="Beispielweg 2, 54321 Beispielstadt"
    ),
    taetigkeit=Taetigkeit(
        beschreibung="Softwareentwicklung",
        startdatum="2025-01-01"
    ),
    leistungserbringung=Leistungserbringung(
        persoenlich=True,
        unterauftrag=False,
        eigene_raeume=True,
        betriebsmittel=False
    ),
    verguetung=Verguetung(
        stundensatz=80.0,
        mehrwertsteuer=True,
        abrechnung_bis=10,
        zahlung_faelligkeit="Monatsende"
    ),
    aufwendungen=Aufwendungen(
        pauschal_abgedeckt=True,
        erstattungsfaehig=False,
        vorherige_zustimmung=True
    ),
    haftung=Haftung(
        haftungsgrenze=5000.0,
        berufshaftpflicht=True
    ),
    konkurrenz=Konkurrenz(
        andere_auftraggeber=True,
        wettbewerber_erlaubt=False,
        vertragsstrafe=10000.0
    ),
    verschwiegenheit=Verschwiegenheit(
        vertragsstrafe=20000.0
    ),
    vertragsdauer=Vertragsdauer(
        beginn="2025-01-01",
        kuendigungsfrist=4,
        kuendigungsdatum="2025-12-31"
    ),
    gerichtsstand=Gerichtsstand(
        erfuellungsort="Musterstadt",
        gerichtsstand="Mustergericht"
    ),
    legal_clauses=LegalClauses(
        content="",
        reasoning=""
    ),
    compliance_check=ComplianceCheck(
        result="",
        reasoning=""
    ),
    formatted_contract=FormattedContract(
        document=""
    )
)

print(vertrag.model_dump_json(indent=4))


### Prompts

In [ ]:
legal_prompts = {
    "system": "Sie sind ein Rechtsexperte, der Klauseln/Absätze von Vertrage über Mitarbeit erstellt oder freie Mitarbeit und deren Bedeutung erläutert.",
    "user": "Rechtsklauseln für freie Mitarbeit Vertrage generieren und deren Anwendung erläutern."
}
compliance_prompts = {
    "system": "Sie prüfen als Experte und Compliance-Inspektor Einhaltung mit  deutschen Rechts dh BGB  Mitarbeitvertage oder freie Mitarbeit Vertrage  und erläutern die Prüfschritte. Sie konnen sich sehr gut umgehen mit Dienstvertrag und Werkvertrag und Unterschied zwinschen.",
    "user": "Überprüfen Sie die Einhaltung des deutschen Rechts, also des BGB, des Arbeitsvertrags oder des freie Mitarbeit und erläutern Sie die weiteren Schritte Ihrer Prüfung"
}
formatting_prompts = {
    "system": f"""
    Sie sind ein Experte für Vertragsgestaltung und Formatierung. Ihre Aufgabe ist es, einen formatierten und vollständigen Vertrag basierend auf der folgenden Vorlage zu erstellen:
    
    {Vertrag_ueber_freie_mitarbeit_v0}
    
    **Anforderungen:**
    1. Halten Sie sich so genau wie möglich an die oben gegebene Struktur.
    2. Ergänzen Sie notwendige Absätze oder Anpassungen, falls sie zur Klarstellung oder zur Einhaltung rechtlicher Vorgaben erforderlich sind.
    3. Lassen Sie an den Stellen, an denen keine Daten verfügbar sind, fünf Zeilen mit Unterstrichen (`_____`) für manuelle Einträge.
    4. Achten Sie darauf, dass alle relevanten Vertragsinformationen enthalten sind und der Vertrag leicht lesbar bleibt.
    """,
    "user": "Formatieren Sie den Vertrag basierend auf der oben bereitgestellten Vorlage und stellen Sie sicher, dass er vollständig ist und alle relevanten rechtlichen Details enthält."
}



## Run all (iterations=3)

In [34]:
# Erstellen Sie einen Vertrag mit Änderungen in drei Iterationen
contract = generate_contract(vertrag, legal_prompts, compliance_prompts, formatting_prompts, iterations=3)
print("wersja-finalowa\n")
print(contract)

🔄 Iteration 1: Refining the contract...

Current Contract
 auftraggeber=Auftraggeber(name='Firma XYZ GmbH', adresse='Musterstraße 1, 12345 Musterstadt', vertreter='Max Mustermann') auftragnehmer=Auftragnehmer(name='John Doe', adresse='Beispielweg 2, 54321 Beispielstadt') taetigkeit=Taetigkeit(beschreibung='Softwareentwicklung', startdatum='2025-01-01') leistungserbringung=Leistungserbringung(persoenlich=True, unterauftrag=False, eigene_raeume=True, betriebsmittel=False) verguetung=Verguetung(stundensatz=Decimal('80.0'), mehrwertsteuer=True, abrechnung_bis=10, zahlung_faelligkeit='Monatsende') aufwendungen=Aufwendungen(pauschal_abgedeckt=True, erstattungsfaehig=False, vorherige_zustimmung=True) haftung=Haftung(haftungsgrenze=Decimal('5000.0'), berufshaftpflicht=True) konkurrenz=Konkurrenz(andere_auftraggeber=True, wettbewerber_erlaubt=False, vertragsstrafe=Decimal('10000.0')) verschwiegenheit=Verschwiegenheit(vertragsstrafe=Decimal('20000.0')) vertragsdauer=Vertragsdauer(beginn='2025-01

#### Dieses muster kann auch verwendet sein

In [ ]:
Vertrag_ueber_freie_mitarbeit_v0 = f"""

Vertrag über freie Mitarbeit

(Bei Anwendung des Musters ist zu prüfen, welche Vertragsbestimmungen übernommen werden wollen. Gegebenenfalls sind Anpassungen und Ergänzungen zu empfehlen.)

Zwischen  
.....................................................................................................................  
(Name und Adresse des Auftraggebers)  

(ggf.: vertreten durch ………………………………….………………………………………………)  
- nachfolgend „Auftraggeber“ genannt -

und

Herrn/Frau .....................................................................................................

wohnhaft .......................................................................................................  
- nachfolgend „Auftragnehmer“ genannt -

wird folgendes vereinbart:

---

**§ 1 Tätigkeit**

Der Auftragnehmer **{contract.taetigkeit.beschreibung}** wird ab dem **{contract.taetigkeit.startdatum}** für den Auftraggeber folgende Tätigkeiten als Auftragnehmer übernehmen: . Ergänzend wird im Einzelfall auf die jeweiligen Auftragsschreiben verwiesen.

Der Auftragnehmer hat die Durchführung und den Ablauf seiner Leistung selbst zu organisieren. Er unterliegt keinen Weisungen des Auftraggebers und ist in der Gestaltung seiner Tätigkeit frei. Auf besondere betriebliche Belange im Zusammenhang mit seiner Tätigkeit ist jedoch Rücksicht zu nehmen.

Der Auftragnehmer ist an keinerlei Vorgaben zum Arbeitsort oder Arbeitszeit gebunden. Projektbezogene Zeitvorgaben des Auftraggebers sind ebenso einzuhalten wie fachliche Vorgaben, soweit diese zur ordnungsgemäßen Vertragsdurchführung erforderlich sind.

Der Auftragnehmer ist ferner berechtigt, Aufträge des Auftraggebers ohne Angaben von Gründen abzulehnen.

Gegenüber den Angestellten des Auftraggebers hat der Auftragnehmer keine Weisungsbefugnis.

---

**§ 2 Leistungserbringung**

Der Auftragnehmer erbringt die Arbeitsleistung in der Regel **{ 'persönlich' if contract.leistungserbringung.persoenlich else 'durch Dritte' }**. Er kann sich zur Erfüllung des Auftrags auch anderer Personen bedienen. Die Hinzuziehung eigener Mitarbeiter oder die Vergabe von Unteraufträgen erfolgt in Abstimmung mit dem Auftraggeber. Für die ordnungsgemäße Erfüllung der vertraglichen Leistungen bleibt er dem Auftraggeber gegenüber verantwortlich. 

Für die steuerlichen und sozialversicherungsrechtlichen Belange hat der freie Mitarbeiter selbst Sorge zu tragen, insbesondere auch für eine angemessene Versicherung für die Altersvorsorge wie auch zum Schutz gegen Krankheiten und den Pflegefall.

Der Auftragnehmer übt seine Tätigkeit in seinen eigenen Räumlichkeiten aus. Soweit in Einzelfällen eine betriebliche Anwesenheit erforderlich wird, stellt der Auftraggeber nach jeweiliger vorheriger Absprache die entsprechenden betrieblichen Einrichtungen zur Verfügung. Der Auftraggeber stellt dem Auftragnehmer alle zur Ausübung seiner Tätigkeiten erforderlichen Informationen, Hilfsmittel und Unterlagen zur Verfügung.

---

**§ 3 Vergütung**

Als Vergütung wird ein Stundenhonorar von **{contract.verguetung.stundensatz} €** zuzüglich der jeweiligen gesetzlichen Mehrwertsteuer vereinbart. Der Auftragnehmer ist verpflichtet, jeweils bis zum Zehnten des Folgemonats eine spezifizierte Abrechnung in Form einer Rechnung zu erstellen.

Das vereinbarte pauschale Honorar wird jeweils am Monatsende fällig. Die Auszahlung erfolgt unbar.

Der Auftragnehmer wird innerhalb von 14 Tagen nach Beginn der Zusammenarbeit dem Auftraggeber ein Konto benennen, auf das das Honorar angewiesen werden kann.

---

**§ 4 Aufwendungsersatz und sonstige Ansprüche**

Mit der Zahlung der in diesem Vertrag vereinbarten Vergütung sind alle Ansprüche des Auftragnehmers gegen den Auftraggeber aus diesem Vertrag erfüllt.

oder

Der Auftragnehmer hat Anspruch auf Ersatz der abgerechneten und nachgewiesenen Aufwendungen, die ihm im Rahmen dieser Vereinbarung in der Ausübung seiner Tätigkeit entstehen. Das Normalmaß erheblich übersteigende Ausgaben werden jedoch nur dann ersetzt, wenn der Auftragnehmer zuvor die Zustimmung des Auftraggebers eingeholt hat. Für die Versteuerung der Vergütung hat der Auftragnehmer selbst zu sorgen.

Der Auftragnehmer wird darauf hingewiesen, dass er nach § 2 Nr. 9 SGB VI rentenversicherungspflichtig sein kann, wenn er auf Dauer und im Wesentlichen nur für einen Auftraggeber tätig ist.

---

**§ 5 Haftung und Gewährleistung**

Sollte der Auftraggeber auf Grund von Leistungen, die vom Auftragnehmer erbracht wurden, in Haftung genommen werden, so verpflichtet sich der Auftragnehmer gegenüber dem Auftraggeber, diesen von derlei Haftung freizustellen.

Für Schäden, die durch Zeitüberschreitung des Auftragnehmers erfolgen, ist die Haftung des Auftragnehmers auf die Höhe von **{contract.haftung.haftungsgrenze if contract.haftung.haftungsgrenze else 'Unbegrenzt'} €** begrenzt. Im Übrigen verpflichtet sich der Auftragnehmer zur kostenlosen Nacharbeit und Beseitigung der von ihm verursachten Mängel.

---

**§ 6 Fortbildungspflicht**

Der Auftragnehmer ist verpflichtet, sich im Rahmen der Durchführung dieses Vertrages auf dem Gebiet seiner Tätigkeit über den aktuellen Entwicklungsstand weiterzubilden und sich über aktuelle Veränderungen auf diesem Gebiet jederzeit auf dem Laufenden zu halten.

---

**§ 7 Konkurrenz**

Der Auftragnehmer darf auch für andere Auftraggeber tätig sein. Will der Auftragnehmer allerdings für einen unmittelbaren Wettbewerber des Auftraggebers tätig werden, bedarf dies der vorherigen schriftlichen Zustimmung des Auftraggebers.

Der Auftragnehmer verpflichtet sich, für jeden Fall der Zuwiderhandlung eine Vertragsstrafe in Höhe von **{contract.konkurrenz.vertragsstrafe} €** zu zahlen.

---

**§ 8 Verschwiegenheit, Aufbewahrung und Rückgabe von Unterlagen**

Die Vertragsparteien verpflichten sich, alle ihnen im Rahmen des Vertrages zugänglich gemachten, sowie bei Gelegenheit der Zusammenarbeit erlangten Informationen über Angelegenheiten der anderen Partei, die als vertraulich gekennzeichnet sind; die bei einer mündlichen Übermittlung als vertraulich bezeichnet werden; oder die aus Sicht eines objektiven Beobachters als vertraulich erkennbar sind; sowie Geschäfts- und Betriebsgeheimnisse vertraulich zu behandeln. Vertrauliche Informationen dürfen ohne schriftliche Einwilligung der anderen Vertragspartei zu einem anderen als dem zur vertragsgemäßen Aufgabenerfüllung vorgesehenen Zweck nicht verwertet, Dritten zugänglich gemacht oder sonst genutzt werden.

Die Parteien tragen dafür Sorge, dass Dritte, derer sie sich als Erfüllungsgehilfen bedienen, ebenfalls die Geheimhaltungspflicht beachten.

Für jeden Fall der schuldhaften Verletzung dieser Verpflichtungen wird eine Vertragsstrafe in Höhe von **{contract.verschwiegenheit.vertragsstrafe} €** vereinbart.

Weitergehender Schadensersatz sowie die Geltendmachung von Unterlassungsansprüchen bleiben vorbehalten.

---

**§ 9 Vertragsdauer und Kündigung**

Der Auftragnehmer nimmt die Tätigkeit am **{contract.vertragsdauer.beginn}** auf.

Das Vertragsverhältnis kann unter Einhaltung einer Frist von **{contract.vertragsdauer.kuendigungsfrist}** Wochen/Monaten zum **{contract.vertragsdauer.kuendigungsdatum}** gekündigt werden. Das Recht zur außerordentlichen Kündigung bleibt hiervon unberührt. Jede Kündigung bedarf zu ihrer Wirksamkeit der Schriftform.

---

**§ 10 Erfüllungsort und Gerichtsstand**

Erfüllungsort und Gerichtsstand ist **{contract.gerichtsstand.erfuellungsort}**.

---

**§ 12 Nebenabreden und salvatorische Klausel**

Nebenabreden und Änderungen des Vertrages bedürfen zu ihrer Wirksamkeit der Schriftform. Dieses Formerfordernis kann weder mündlich noch stillschweigend aufgehoben oder außer Kraft gesetzt werden.

Die teilweise oder vollständige Unwirksamkeit einzelner Bestimmungen dieses Vertrages berührt nicht die Wirksamkeit der übrigen Regelungen des Vertrages.

---

**§ 13 Vertragsaushändigung**

Jede der Vertragsparteien hat eine schriftliche Ausfertigung dieses Vertrages erhalten.

---

Ort, Datum: **__________**

**Unterschrift Auftraggeber**  
______________________  

**Unterschrift Auftragnehmer**  
______________________  

"""

### First generation-before-saving

In [35]:
response= (""" Current Contract
 auftraggeber=Auftraggeber(name='Firma XYZ GmbH', adresse='Musterstraße 1, 12345 Musterstadt', vertreter='Max Mustermann') auftragnehmer=Auftragnehmer(name='John Doe', adresse='Beispielweg 2, 54321 Beispielstadt') taetigkeit=Taetigkeit(beschreibung='Softwareentwicklung', startdatum='2025-01-01') leistungserbringung=Leistungserbringung(persoenlich=True, unterauftrag=False, eigene_raeume=True, betriebsmittel=False) verguetung=Verguetung(stundensatz=Decimal('80.0'), mehrwertsteuer=True, abrechnung_bis=10, zahlung_faelligkeit='Monatsende') aufwendungen=Aufwendungen(pauschal_abgedeckt=True, erstattungsfaehig=False, vorherige_zustimmung=True) haftung=Haftung(haftungsgrenze=Decimal('5000.0'), berufshaftpflicht=True) konkurrenz=Konkurrenz(andere_auftraggeber=True, wettbewerber_erlaubt=False, vertragsstrafe=Decimal('10000.0')) verschwiegenheit=Verschwiegenheit(vertragsstrafe=Decimal('20000.0')) vertragsdauer=Vertragsdauer(beginn='2025-01-01', kuendigungsfrist=4, kuendigungsdatum='2025-12-31') gerichtsstand=Gerichtsstand(erfuellungsort='Musterstadt', gerichtsstand='Mustergericht') legal_clauses=LegalClauses(content='§ 1 Mietgegenstand und Gebrauch: Der Vermieter vermietet an den Mieter die im Mietvertrag näher bezeichnete Wohnung nebst den zugehörigen Nebenräumen und Einrichtungen. Der Mieter ist berechtigt, die Wohnung gemäß dem vertraglich vereinbarten Zweck, nämlich zu Wohnzwecken, zu nutzen. Jede andere Nutzung der Wohnung bedarf der ausdrücklichen schriftlichen Zustimmung des Vermieters.', reasoning='Diese Klausel definiert den Mietgegenstand klar und grenzt den Gebrauch auf den vertraglich vereinbarten Zweck ein, um potenzielle Konflikte über die Nutzung der Mieträumlichkeiten zu vermeiden. Sie stellt sicher, dass der Mieter die Wohnung ausschließlich zu Wohnzwecken nutzt und jede andere Nutzung, die möglicherweise der Erlaubnis durch den Vermieter bedarf, im Voraus geklärt wird. Dies schützt die Rechte des Vermieters hinsichtlich der Art und Weise, wie das Eigentum genutzt wird, und minimiert das Risiko von Missverständnissen oder illegalen Nutzungsarten, die die jeweilige Mietvereinbarung beeinträchtigen könnten.') compliance_check=ComplianceCheck(result='Die Prüfung ergab, dass der Vertrag in Übereinstimmung mit dem deutschen Recht, insbesondere dem BGB, steht.', reasoning='1. **Vertragsart bestimmen:** Zunächst wird festgestellt, ob es sich um einen Arbeitsvertrag oder einen freien Mitarbeitvertrag handelt. Dies ist entscheidend, da unterschiedliche gesetzliche Regelungen gemäß dem BGB Anwendung finden. Ein Arbeitsvertrag ist gekennzeichnet durch eine weisungsgebundene, fremdbestimmte Tätigkeit in persönlicher Abhängigkeit (§ 611a BGB), wohingegen ein freier Mitarbeitervertrag mehr Gestaltungsfreiheit und Selbstständigkeit bietet (§ 611 BGB). \n\n2. **Vertragsinhalte prüfen:** Im nächsten Schritt wird der Vertrag auf notwendige Bestandteile geprüft. Bei einem Arbeitsvertrag sind wesentliche Punkte wie Arbeitsort, Arbeitszeit, Vergütung, Urlaubsanspruch und Kündigungsfristen zu überprüfen (§ 611a BGB). Für freie Mitarbeitverträge sind insbesondere die Regelungen zur Vergütung und zur Art und Umfang der zu erbringenden Leistungen bedeutend (§ 631 BGB). \n\n3. **Prüfung der Einhaltung von Schutzvorschriften:** Im Falle des Arbeitsvertrags sind zwingende arbeitsrechtliche Schutzvorschriften wie das Mindestlohngesetz, Arbeitszeitgesetz und Kündigungsschutzgesetz zu beachten, die den Arbeitnehmer schützen sollen. Bei freien Mitarbeitern sind solche Schutzvorschriften nicht zwingend anwendbar, dennoch sollten Regelungen zur Vermeidung einer Scheinselbstständigkeit und Sozialversicherungspflichtigkeit beachtet werden.\n\n4. **Konsistenz und Vollständigkeit prüfen:** Schließlich wird überprüft, ob der Vertrag konsistent und vollständig ist, d.h. dass er alle notwendigen Klauseln und keine widersprüchlichen Regelungen enthält. Der Vertrag sollte auch keine Formvorschriften verletzen, wie z.B. das Schriftformerfordernis bei befristeten Arbeitsverträgen (§ 14 TzBfG).\n\nZusammenfassend ist die Einhaltung der gesetzlichen Anforderungen des BGB für die jeweiligen Vertragsarten gegeben. Diese Prüfung deckt alle wesentlichen Punkte ab, die für die Rechtssicherheit des Vertrags erforderlich sind.') formatted_contract=FormattedContract(document='FREIER MITARBEITER VERTRAG\n\nZwischen\n\n_____ (Name des Auftraggebers)\n_____ (Adresse des Auftraggebers)\n_____ (Kontaktinformationen des Auftraggebers)\n\nund\n\n_____ (Name des freien Mitarbeiters)\n_____ (Adresse des freien Mitarbeiters)\n_____ (Kontaktinformationen des freien Mitarbeiters)\n\nwird folgender Vertrag über freie Mitarbeit geschlossen:\n\n§ 1 Vertragsgegenstand\nDer Auftraggeber verpflichtet den freien Mitarbeiter zur Erbringung folgender Dienstleistungen: \n_____\n_____\n\n§ 2 Vertragslaufzeit\nDer Vertrag beginnt am _____ und endet am _____. Eine Verlängerung des Vertrages bedarf der schriftlichen Zustimmung beider Parteien.\n\n§ 3 Vergütung\nDie Vergütung beträgt _____ Euro pro Stunde/Projekt und ist nach folgender Modalität zu zahlen: _____\n\n§ 4 Arbeitszeiten\nDie Arbeitszeiten richten sich nach: _____\n\n§ 5 Kündigung\nDer Vertrag kann beidseitig unter Einhaltung einer Kündigungsfrist von _____ Wochen/Monaten gekündigt werden.\n\n§ 6 Vertraulichkeit\nDer freie Mitarbeiter verpflichtet sich, alle während der Zusammenarbeit erlangten Informationen vertraulich zu behandeln und nicht an Dritte weiterzugeben.\n\n§ 7 Sonstiges\nWeitere Vereinbarungen: _____\n\nSollte eine Bestimmung dieses Vertrages unwirksam sein oder werden, so bleiben die übrigen Bestimmungen davon unberührt.\n\nOrt: _____\nDatum: _____\n\nUnterschrift Auftraggeber: ____________________\n\nUnterschrift freier Mitarbeiter: ____________________')
Contract saved to Firma_XYZ_GmbH_2025-02-10_11-27-38.md""")

In [36]:
print(response)

 Current Contract
 auftraggeber=Auftraggeber(name='Firma XYZ GmbH', adresse='Musterstraße 1, 12345 Musterstadt', vertreter='Max Mustermann') auftragnehmer=Auftragnehmer(name='John Doe', adresse='Beispielweg 2, 54321 Beispielstadt') taetigkeit=Taetigkeit(beschreibung='Softwareentwicklung', startdatum='2025-01-01') leistungserbringung=Leistungserbringung(persoenlich=True, unterauftrag=False, eigene_raeume=True, betriebsmittel=False) verguetung=Verguetung(stundensatz=Decimal('80.0'), mehrwertsteuer=True, abrechnung_bis=10, zahlung_faelligkeit='Monatsende') aufwendungen=Aufwendungen(pauschal_abgedeckt=True, erstattungsfaehig=False, vorherige_zustimmung=True) haftung=Haftung(haftungsgrenze=Decimal('5000.0'), berufshaftpflicht=True) konkurrenz=Konkurrenz(andere_auftraggeber=True, wettbewerber_erlaubt=False, vertragsstrafe=Decimal('10000.0')) verschwiegenheit=Verschwiegenheit(vertragsstrafe=Decimal('20000.0')) vertragsdauer=Vertragsdauer(beginn='2025-01-01', kuendigungsfrist=4, kuendigungsdat